<a href="https://colab.research.google.com/github/Gooogr/Brain2Image/blob/master/sub_notebooks/PyTorch_DGAN_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on: <br>
https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html<br>
https://github.com/nashory/gans-awesome-applications